In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from datetime import date
import joblib

# Model Training

The cell below shows the code used to train RF-Seward. Note that X (input features) and y (actual snow depths) datasets are not provided in this GitHub repository, but correspond to the DTP data described in the related manuscript (see readme file for more details).

In [2]:
# RF-Seward training code 
rf_best = RandomForestRegressor(bootstrap = True, max_depth = 10, max_features =2, max_samples = 0.8, n_estimators = 70, min_samples_leaf = 1, min_samples_split=15)
rf_best.fit(X, y)

NameError: name 'X' is not defined

# Input Data Cleaning

The code presented here was designed for pulling features from the NGEE-Arctic iButton datasets publically available on ESS-Dive (see readme file for more details). 

In [3]:
# Change date from string to datetime object
# DATA is an array of string dates, STRING is the string defining the date pattern
# RETURNS an array of datetime objects 
def changedate(data, string):
    return [datetime.strptime(i, string) for i in data]

# Function to rename columns in a dataframe 
# DATA is the dataframe; OLDS are the current column names; NEWS are the desired column names
# OLDs and NEWs can either be arrays of multipe names or single strings 
# RETURNS an updated dataframe with renamed columns 
def renamecols(data, olds, news):
    if type(olds) == str: 
        return data.rename({olds: news}, axis = 1) 
    num = len(olds)
    for i in np.arange(num):
        data = data.rename({olds[i]: news[i]}, axis =1)
    return data 

# Create an array of dates from dataframe RAWDATA by 
# pulling date from the existing COLNAME (default = 'DateTime') column.
# RETURNS an array of dates. 
def add_date(rawdata, colname = 'DateTime'):
    return [i.date() for i in rawdata[colname]]

# Function to determine which time interval a data point belongs to. 
# TIME is a datetime and INTERVAL is the time interval that 
# RF-Seward features are calculated from. We use 4-hour intervals 
# because the iButton data we had was collected in 4-hour intervals.
# This function was originally created to aggregate the training data from 
# 30-minute intervals to 4-hour intervals and is a little bit 
# unnecessary here. 
# RETURNS which time interval (or group) a datetime belongs to. 
def get_group(time, interval):
    hours = np.arange(interval, 25, interval)
    num = 1
    hour = time.hour
    for h in hours:
        if hour < h: 
            return num 
        num = num + 1  
        
# RETURNS an array of 'groups' given a dataframe DATA, the desired
# time interval INTERVAL, and the name of the DateTime column (COLNAME).
def assign_group(data, interval, colname = 'DateTime'): 
    return [get_group(i, 4) for i in data[colname]]

# RETURNS a cleaned DataFrame with all features used by RF-Seward given the RAWDATA. 
def iButton_clean(rawdata):
    all_data = pd.DataFrame([])
    rawdata = rawdata.copy()
    rawdata = renamecols(rawdata, 'Temperature', 'GST')
    try: 
        rawdata['DateTime'] = changedate(rawdata['Date_Time'], '%m/%d/%Y %H:%M:%S')
    except:
        try:
            rawdata['DateTime'] = changedate(rawdata['Date_Time'], '%m/%d/%y %H:%M')
        except:
            rawdata['DateTime'] = changedate(rawdata['Date_Time'], '%Y-%m-%d %H:%M:%S')
    rawdata = rawdata.drop('Date_Time', axis = 1)
    rawdata['day'] = add_date(rawdata)
    # The next 2 lines are little bit redundant because the data is already in 4-hour intervals 
    rawdata['hour_ind'] = assign_group(rawdata, 4)
    rawdata = rawdata.groupby(['hour_ind', 'iButton', 'day']).mean()
    rawdata = rawdata.reset_index()
    rawdata = rawdata.drop('DateTime', axis = 1)
    buttons = np.unique(rawdata['iButton'])
    # The for loop below creates features for each iButton at a time and then combines 
    # everything into a single dataset 
    for  b in buttons:
        data = rawdata.loc[rawdata['iButton']==b]
        data = data.drop('iButton', axis = 1)
        datam = create_features(data)
        datam['iButton'] = b
        all_data = pd.concat([all_data, datam])
    return all_data


# RETURNS the mean, max, min, and standard deviation of DATA when grouped by COLS.
def grouped_data(data, cols): 
    datamean = data.groupby(cols).mean()
    datamax = data.groupby(cols).max()
    datamin = data.groupby(cols).min()
    datastd = data.groupby(cols).std()
    datamean = datamean.reset_index()
    datamax = datamax.reset_index()
    datamin = datamin.reset_index() 
    datastd = datastd.reset_index()
    return datamean, datamax, datamin, datastd 

# RETURNS a dataframe of features required by RF-SEWARD given DATA, 
# the window-size (WSIZE) for the features window-before, window-surrounding,
# and 'window-after' (see readme), and the name of the 'day' column (COLS, default = 'day')
def create_features(data, wsize = 30, cols = 'day'): 
    # Creating grouped datasets to get min, max, mean, stds 
    datamean, datamax, datamin, datastd = grouped_data(data, cols)
    # Creating GST range feature 
    min_max_range = datamax['GST'] - datamin['GST']
    # Initiating dataframe with features 
    datam = pd.DataFrame([])
    datam['time'] = datamax.day
    # Adding in GST_range and Max_GST as features 
    datam['GST_range'] = min_max_range 
    datam['Max_GST'] = datamax['GST']
    datam['Min_GST'] = datamin['GST']
    # Adding in window before feature
    # Adding in nan data entries for days where no data was recorded 
    # because otherwise consecutive data entries would be aggregated 
    # over even if they weren't neighboring days.
    dates = pd.date_range(datastd.day.values[0], datastd.day.values[-1])
    datestable = pd.DataFrame([])
    datestable['day'] = [i.date() for i in dates]
    datastd = datastd.merge(datestable, how = 'outer', on = 'day')
    window = [np.mean(np.array(datastd['GST'][i - wsize: i])) for i in np.arange(wsize, len(datastd))]
    window = np.append(np.ones(wsize) * np.nan, window)
    datastd['window'] = window
    datam['GST_wk_prior'] = datastd['window']
    # Adding GST_std and GST_av features, even though tese aren't used 
    # in the final model
    datam['GST_std'] = datastd['GST']
    datam['GST_av'] = datamean['GST']
    #Adding in window after feature 
    window = [np.mean(np.array(datastd['GST'][i + 1: i + wsize + 1])) for i in np.arange(0, len(datastd) - wsize)]
    window = np.append(window, np.ones(wsize) * np.nan)
    datastd['window'] = window
    datam['GST_wk_after'] = datastd['window']
    #Adding in window surrounding feature 
    temp = int((wsize - 1)/2)
    window = [np.mean(np.array(datastd['GST'][i - temp: i + temp + 1])) for i in np.arange(temp, len(datastd) - temp)]
    window = np.append(window, np.ones(temp) * np.nan)
    window = np.append(np.ones(temp) * np.nan, window)
    datastd['window'] = window
    datam['GST_wk_sur'] = datastd['window']
    return datam 


In [4]:
# Reading in iButton data 
iButton_data = pd.read_csv('../iButtons2022/2021_2022_TL27_iButton_snow_temperatures.csv')
# Creating iButton features 
iButton_data = iButton_data.loc[:, ['iButton', 'Date_Time', 'Temperature']]
iButton_features = iButton_clean(iButton_data)
# List of features used in the final model
cols = [ 'GST_wk_after', 'GST_range', 'GST_wk_prior', 'GST_wk_sur', 'Max_GST']
# Dropping nans because the random forest model can not handle nans 
iButton_features = iButton_features.dropna()

/var/folders/nc/jk9598l14tsb028s9yx9d10m00162w/T/ipykernel_10597/1148948345.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  iButton_data = pd.read_csv('../iButtons2022/2021_2022_TL27_iButton_snow_temperatures.csv')


# Generating Predictions

In [5]:
# Read in joblib random forest
rf_seward = joblib.load('rf_seward.joblib')

In [6]:
# Generate predictions from the features calculated in the previous section
snow_pred = rf_seward.predict(iButton_features.loc[:, cols])